In [2]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu116.html
     |████████████████████████████████| 8.0 MB 2.9 MB/s 
     |████████████████████████████████| 3.5 MB 47.3 MB/s 
     |████████████████████████████████| 2.4 MB 556 kB/s 
     |████████████████████████████████| 706 kB 726 kB/s 
     |████████████████████████████████| 407 kB 4.0 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=123a570bd63fac88c206db6b627aae307062a208801830d178243a7f0642bedc
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [16]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv


In [11]:
edge_idx = torch.tensor([
                         [0, 1], 
                         [1, 0], 
                         [1, 2],
                         [2, 1]
                        ], 
                        dtype=torch.long)

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_idx.t().contiguous())
print(data)
print(f'data.x = 3 nodes each with 1 feature')
print(f'data.edge_index --> 4/2 = 2 undirected edges')
print(data.is_directed())
print(data.has_self_loops())

Data(x=[3, 1], edge_index=[2, 4])
data.x = 3 nodes each with 1 feature
data.edge_index --> 4/2 = 2 undirected edges
False
False


In [19]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(dataset)

Cora()


In [20]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_idx = data.x, data.edge_index
        
        x = self.conv1(x, edge_idx)
        x = F.relu()
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_idx)
        x = F.log_softmax(x, dim=1)

        return x

